In [1]:
import tensorflow as tf
import tensorflow_text as tf_text
#pip install -U tensorflow-text==2.6.0

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.6.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
from tensorflow.python.platform import gfile
from itertools import chain

In [3]:
from glob import glob
import requests
import sentencepiece as spm
from regexTokenizer import regexKmerTokenizer


In [4]:
tok = regexKmerTokenizer(1000, overlapping= False, protein = True)
%time df = tok.seq2pandas('human genome/GRCh38_latest_protein.faa.gz', docformat = 'fasta',from_file=True)
def sppliter(name):
    left, right = name.find(' '), name.rfind('[')
    return [name[:left], name[left:right], name[right:]]
df[['ID', 'name', 'species']] = [sppliter(i) for i in df.name]
df


CPU times: total: 12.6 s
Wall time: 12.6 s


,name,sequence,ID,species
0,alpha-2-macroglobulin isoform a precursor,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,NP_000005.3,[Homo sapiens]
1,arylamine N-acetyltransferase 2,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,NP_000006.2,[Homo sapiens]
2,"medium-chain specific acyl-CoA dehydrogenase,...",MAAGFGRCCRVLRSISRFHWRSQHTKANRQREPGLGFSFEFTEQQK...,NP_000007.1,[Homo sapiens]
3,"short-chain specific acyl-CoA dehydrogenase, ...",MAAALLARASGPARRALCPRAWRQLHTIYQSVELPETHQMLLQTCR...,NP_000008.1,[Homo sapiens]
4,very long-chain specific acyl-CoA dehydrogena...,MQAARMAASLGRQLLRLGGGSSRLTALLGQPRPGPARRPYAGGAAQ...,NP_000009.1,[Homo sapiens]
...,...,...,...,...
115346,NADH dehydrogenase subunit 4L (mitochondrion),MPLIYMNIMLAFTISLLGMLVYRSHLMSSLLCLEGMMLSLFIMATL...,YP_003024034.1,[Homo sapiens]
115347,NADH dehydrogenase subunit 4 (mitochondrion),MLKLIVPTIMLLPLTWLSKKHMIWINTTTHSLIISIIPLLFFNQIN...,YP_003024035.1,[Homo sapiens]
115348,NADH dehydrogenase subunit 5 (mitochondrion),MTMHTTMTTLTLTSLIPPILTTLVNPNKKNSYPHYVKSIVASTFII...,YP_003024036.1,[Homo sapiens]
115349,NADH dehydrogenase subunit 6 (mitochondrion),MMYALFLLSVGLVMGFVGFSSKPSPIYGGLVLIVSGVVGCVIILNF...,YP_003024037.1,[Homo sapiens]


In [5]:
import numpy as np
df.sequence.apply(len).describe(percentiles=list(np.linspace(0.9,1,21)))

count    115351.000000
mean        660.229049
std         741.071372
min          25.000000
50%         489.000000
90%        1264.000000
90.5%      1292.000000
91%        1322.000000
91.5%      1359.000000
92%        1400.000000
92.5%      1437.000000
93%        1478.000000
93.5%      1521.000000
94%        1575.000000
94.5%      1630.000000
95%        1693.000000
95.5%      1761.000000
96%        1839.000000
96.5%      1922.000000
97%        2039.000000
97.5%      2181.250000
98%        2353.000000
98.5%      2567.000000
99%        2977.000000
99.5%      4220.500000
100%      35991.000000
max       35991.000000
Name: sequence, dtype: float64

In [ ]:
%time spm.SentencePieceTrainer.train(sentence_iterator=iter(df.name), model_prefix='humanProtNames',\
                                     vocab_size=10000, hard_vocab_limit = False, max_sentencepiece_length =30, split_by_number = False, 
                                     split_by_unicode_script = False)

In [7]:
%time spm.SentencePieceTrainer.train(sentence_iterator=iter(df.sequence), model_prefix='humanProtAA', \
                                     vocab_size=30000,  max_sentence_length = 36000, max_sentencepiece_length =30, \
                                     hard_vocab_limit = False, train_extremely_large_corpus= True,\
                                     pad_id =-1, bos_piece = 'bos',  eos_piece = 'eos') #bos_id = -1, eos_id = -1,

CPU times: total: 49min 20s
Wall time: 6min 36s


In [ ]:
df

In [8]:
sp = spm.SentencePieceProcessor(model_file='humanProtNames.model')

In [11]:
sp('NADH 3.')

TypeError: 'SentencePieceProcessor' object is not callable

In [ ]:
sp.encode_as_ids('NADH 3.')

In [ ]:
encoded_input = sp.Encode(df.sequence.iloc[3][10:1000])
print(encoded_input)

tokenized_input = [sp.IdToPiece(id) for id in encoded_input]
print(*tokenized_input)

In [ ]:
model = gfile.GFile('humanProtNames.model', 'rb').read()
SPT = tf_text.SentencepieceTokenizer(model=model, out_type=tf.string, nbest_size=0, alpha=1.0)


In [ ]:
df.sequence.iloc[3][10:1000]

In [ ]:
SPT.tokenize('NADH ')

In [ ]:
SPT.detokenize(SPT.tokenize(df.sequence.iloc[3][10:1000]))

In [ ]:
from itertools import chain
%time biglist = list(chain.from_iterable(tok(df.sequence[:100])))

--help (show help)  type: bool default: false
--version (show version)  type: bool default: false
--minloglevel (Messages logged at a lower level than this don't actually get logged anywhere)  type: int default: 0
--input (comma separated list of input sentences)  type: std::string default: ""
--input_format (Input format. Supported format is `text` or `tsv`.)  type: std::string default: ""
--model_prefix (output model prefix)  type: std::string default: ""
--model_type (model algorithm: unigram, bpe, word or char)  type: std::string default: "unigram"
--vocab_size (vocabulary size)  type: int32 default: 8000
--accept_language (comma-separated list of languages this model can accept)  type: std::string default: ""
--self_test_sample_size (the size of self test samples)  type: int32 default: 0
--character_coverage (character coverage to determine the minimum symbols)  type: double default: 0.9995
--input_sentence_size (maximum size of sentences the trainer loads)  type: int32 default: 0
--shuffle_input_sentence (Randomly sample input sentences in advance. Valid when --input_sentence_size > 0)  type: bool default: true
--seed_sentencepiece_size (the size of seed sentencepieces)  type: int32 default: 1000000
--shrinking_factor (Keeps top shrinking_factor pieces with respect to the loss)  type: double default: 0.75
--num_threads (number of threads for training)  type: int32 default: 16
--num_sub_iterations (number of EM sub-iterations)  type: int32 default: 2
--max_sentencepiece_length (maximum length of sentence piece)  type: int32 default: 16
--max_sentence_length (maximum length of sentence in byte)  type: int32 default: 4192
--split_by_unicode_script (use Unicode script to split sentence pieces)  type: bool default: true
--split_by_number (split tokens by numbers (0-9))  type: bool default: true
--split_by_whitespace (use a white space to split sentence pieces)  type: bool default: true
--split_digits (split all digits (0-9) into separate pieces)  type: bool default: false
--treat_whitespace_as_suffix (treat whitespace marker as suffix instead of prefix.)  type: bool default: false
--control_symbols (comma separated list of control symbols)  type: std::string default: ""
--user_defined_symbols (comma separated list of user defined symbols)  type: std::string default: ""
--required_chars (UTF8 characters in this flag are always used in the character set regardless of --character_coverage)  type: std::string default: ""
--byte_fallback (decompose unknown pieces into UTF-8 byte pieces)  type: bool default: false
--vocabulary_output_piece_score (Define score in vocab file)  type: bool default: true
--normalization_rule_name (Normalization rule name. Choose from nfkc or identity)  type: std::string default: "nmt_nfkc"
--normalization_rule_tsv (Normalization rule TSV file. )  type: std::string default: ""
--denormalization_rule_tsv (Denormalization rule TSV file.)  type: std::string default: ""
--add_dummy_prefix (Add dummy whitespace at the beginning of text)  type: bool default: true
--remove_extra_whitespaces (Removes leading, trailing, and duplicate internal whitespace)  type: bool default: true
--hard_vocab_limit (If set to false, --vocab_size is considered as a soft limit.)  type: bool default: true
--use_all_vocab (If set to true, use all tokens as vocab. Valid for word/char models.)  type: bool default: false
--unk_id (Override UNK (<unk>) id.)  type: int32 default: 0
--bos_id (Override BOS (<s>) id. Set -1 to disable BOS.)  type: int32 default: 1
--eos_id (Override EOS (</s>) id. Set -1 to disable EOS.)  type: int32 default: 2
--pad_id (Override PAD (<pad>) id. Set -1 to disable PAD.)  type: int32 default: -1
--unk_piece (Override UNK (<unk>) piece.)  type: std::string default: "<unk>"
--bos_piece (Override BOS (<s>) piece.)  type: std::string default: "<s>"
--eos_piece (Override EOS (</s>) piece.)  type: std::string default: "</s>"
--pad_piece (Override PAD (<pad>) piece.)  type: std::string default: "<pad>"
--unk_surface (Dummy surface string for <unk>. In decoding <unk> is decoded to `unk_surface`.)  type: std::string default: " ⁇ "
--train_extremely_large_corpus (Increase bit depth for unigram tokenization.)  type: bool default: false